In [ ]:
import pandas as pd
import multiprocessing as mp
from nltk.tokenize import word_tokenize
from functools import partial
from tqdm import tqdm

In [ ]:
books = pd.read_csv("book_summaries.csv", index_col=False)

In [ ]:
books["summary"] = books["summary"].apply(word_tokenize)

In [ ]:
len(books["summary"])

In [ ]:
# def calculate_jaccard_similarity(word_tokens1, word_tokens2):
#     # Combine both tokens to find union.
#     both_tokens = word_tokens1 + word_tokens2
#     union = set(both_tokens)

#     # Calculate intersection.
#     intersection = set()
#     for w in word_tokens1:
#         if w in word_tokens2:
#             intersection.add(w)

#     jaccard_score = len(intersection)/len(union)
#     return jaccard_score

iteration = 0

def calculate_jaccard_similarity(word_tokens1, word_tokens2):
    set1 = set(word_tokens1)
    set2 = set(word_tokens2)

    intersection = set1 & set2
    union = set1 | set2

    jaccard_score = len(intersection) / len(union)
    
    iteration += 1
    print(iteration)
    
    return jaccard_score

def process_pair(pair, summaries, isbn_list):
    idx1, idx2 = pair
    isbn1 = isbn_list[idx1]
    isbn2 = isbn_list[idx2]
    summary1 = summaries.loc[idx1]
    summary2 = summaries.loc[idx2]
    similarity = calculate_jaccard_similarity(summary1, summary2)
 
    return isbn1, isbn2, similarity

def calculate_similarity_parallel(summaries, isbn_list):
    pairs = [(i, j) for i in summaries.index for j in summaries.index if i < j]
    pool = mp.Pool()
    func = partial(process_pair, summaries=summaries, isbn_list=isbn_list)
    results = list(tqdm(pool.imap(func, pairs), total=len(pairs)))
    pool.close()
    pool.join()
    return results

In [ ]:
results = calculate_similarity_parallel(books['summary'], books["isbn"])